In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 22 17:22:54 2018

@author: WUHUIHANG915
"""

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl 

from pandas.tseries.offsets import DateOffset
# 设置中文编码和负号的正常显示
plt.rcParams['font.sans-serif'] = 'simhei'
plt.rcParams['axes.unicode_minus'] = False
#最小二乘
import statsmodels.api as sm 
# Wind导入数据设置            
from WindPy import w
import time
import re
import datetime
w.start()


def get_WD_data(ind='',start_data='',end_data='',name=''):
    temp = w.edb(ind, start_data, end_data,"Fill=Previous")
    temp_data = pd.DataFrame()
    temp_data['time'] = temp.Times    
    temp_data[name] =  temp.Data[0]  
    temp_data['time']=pd.to_datetime(temp_data['time'])
    temp_data = temp_data.set_index('time')
    temp_data['time'] = temp_data.index.strftime('%Y-%m')
    return temp_data


def two_plot_fig(df='',ind1='',ind2='',date='',num1='',num2='',num3='',num4=''):
    temp_data =df.copy() 
    temp_data['time'] = pd.to_datetime(temp_data['time'])
    fig_x = temp_data[date]
    fig_y1= temp_data[ind2] 
    fig_y2= temp_data[ind1] 
    fig = plt.figure(figsize=(10,6)) 
    plt.style.use('ggplot') 
    # 设置图框的大小
    fig = plt.figure(figsize=(16,9))
    # 绘图--1
    ax1 = fig.add_subplot(111)
    ax1.plot(fig_x, # x轴数据
             fig_y1, # y轴数据
             linestyle = '-', # 折线类型
             linewidth = 2, # 折线宽度
             color = 'steelblue', # 折线颜色
             marker = 'o', # 点的形状
             markersize = 6, # 点的大小
             markeredgecolor='black', # 点的边框色
             markerfacecolor='steelblue', # 点的填充色
             label = ind2 +'(左轴，单位%)') # 添加标签
        # 添加阴影区间
    ax1.axhline(np.mean(fig_y1),label='均值')
    ax1.axhline(np.mean(fig_y1)+np.std(fig_y1),linestyle = '-.',label='上1倍标准差')
    ax1.axhline(np.mean(fig_y1)-np.std(fig_y1),linestyle = '-.',label='下1倍标准差')
    plt.legend(loc='upper right', bbox_to_anchor=(1,0.95),ncol=2,fancybox=True,shadow=True)
    ax2 = ax1.twinx()

    # 绘图--2
    ax2.plot(fig_x, # x轴数据
             fig_y2, # y轴数据
             linestyle = '-', # 折线类型
             linewidth = 2, # 折线宽度
             color = '#ff9999', # 折线颜色
              marker = 'o', # 点的形状
              markersize = 6, # 点的大小
              markeredgecolor='black', # 点的边框色
              markerfacecolor='#ff9999', # 点的填充色
             label = ind1 + '(右轴，单位%)') # 添加标签
    ax2.grid(False)
    # 添加标题和坐标轴标签
    plt.axvspan(fig_x[num1] , fig_x[num2], facecolor='gray', alpha=0.3)
    plt.axvspan(fig_x[num3] , fig_x[num4], facecolor='gray', alpha=0.3)   
    plt.title(ind1 + '和' +ind2+'相关关系图')
    plt.tick_params(top = 'off', right = 'off')
    # 获取图的坐标信息
    ax = plt.gca()
    # 设置日期的显示格式  
    date_format = mpl.dates.DateFormatter('%Y-%m')  
    ax.xaxis.set_major_formatter(date_format) 
    # 设置x轴每个刻度的间隔天数
    xlocator = mpl.ticker.MultipleLocator(360)
    ax.xaxis.set_major_locator(xlocator)    
    # 为了避免x轴日期刻度标签的重叠，设置x轴刻度自动展现，并且45度倾斜
    fig.autofmt_xdate(rotation = 45)   
    # 显示图例
    plt.legend(loc='upper right', bbox_to_anchor=(1,0.87),ncol=2,fancybox=True,shadow=True)
    # 显示图形
    plt.show()
    
def lead_cor_picture(factor_x = '',factor_y = '',lead_max_month = '',gap=''):
    x_table = factor_x
    y_table = factor_y
    result = pd.DataFrame()
    for y_col_name in y_table.columns.drop('time'):
        for x_col_name in x_table.columns.drop('time'):
            for month_diff in range(lead_max_month+1):
                if month_diff != 0:
                    x_table['time'] = x_table['time'].shift(gap)
                temp = pd.merge(x_table, y_table,how = 'inner', on = 'time')
                xy_cor = temp[y_col_name].corr(temp[x_col_name])
                temp_result = pd.DataFrame({'Y_值':y_col_name,'X_值':x_col_name,
                                       '提前量': month_diff,'相关系数':xy_cor},index=['index'])
                result = temp_result.append(result)
                result.to_excel('result.xlsx')
                print(result)     
                
                
def RLM_model(df=''):
#获得汇总信息
    x=sm.add_constant(df[factor_X])
    #线性回归增加常数项 y=kx+b
    y=df[factor_Y]
    model=sm.RLM(y,x)
    results=model.fit()
    print(results.summary())
    print('Parameters: ', results.params)
    print('P值: ', results.pvalues)    

##根据领先性水平，调整X
def adj_data(df='',gap='',month_gap=''):
    X = merge_data.loc[:,['time',factor_X]]
    Y = merge_data.loc[:,['time',factor_Y]]
    X['time1'] = X['time']
    X['time1'] = X['time1']+DateOffset(months=month_gap)
    X['time'] = X['time'].shift(gap)   
    for i in range(len(X['time'])):
        if str(X['time'][i])=='NaT':
            X['time'][i]=X['time1'][i]
    del X['time1']
    temp = pd.merge(X, Y,how = 'outer', on = 'time')
    print(temp[factor_X].corr(temp[factor_Y]))
    return temp

def adj_data1(df='',gap='',month_gap=''):
    X = merge_data.loc[:,['time',factor_X]]
    Y = merge_data.loc[:,['time',factor_Y]]
    X['time'] = X['time'].shift(gap)
    temp = pd.merge(X, Y,how = 'inner', on = 'time')
    return temp   



def year_growth(df = '',ind = ''):
    arr_x = df[ind]
    Is = []
    for i in range(len(arr_x)):
        if i==0:
            Is.append(np.nan)
        else:
            Is.append((arr_x[i]+arr_x[i-1])/arr_x[i-1])
    df[ind+'增长率'] = Is
    return df


# 将月度数据转换成季度数据
def transdate_q_m(df=''):
    temp_VA_q = df.copy()
    temp_VA_q['time'] = pd.to_datetime(temp_VA_q['time'])
    temp_VA_q = temp_VA_q.set_index('time')
    temp_VA_q = temp_VA_q.resample('Q').mean()
    temp_VA_q['time'] = temp_VA_q.index.strftime('%Y-%m') 
    df = temp_VA_q 
    return df


def mean_Imputation(df='',ind=''):
    arr_x = df[ind]
    Is = []
    for i in range(len(arr_x)):
        if i==0:
           Is.append(np.nan)
        else:
            if arr_x[i]==np.nan:
               Is.append((arr_x[i-1]+arr_x[i+1])/2) 
            else:
               Is.append(arr_x[i])
    df[ind] = Is
    return df
   
#从累计值换算成累计同比的函数

#def agg_growth(df = '',ind = '',date=''):
#    arr_x = df[ind]
#    arr_q = df[date].index.strftime('%m')
#    Is = []
#    Is_3M = []
#    for i in range(len(arr_x)):
#        if i==0:
#            Is.append(np.nan)
#        else:
#            if arr_q[i] == 3:
#               Is_3M.append(arr_x[i])
#            else:
#               Is.append((arr_x[i]+arr_x[i-1])/arr_x[i-1])
#    df[ind+'增长率'] = Is
#    return df   




ModuleNotFoundError: No module named 'WindPy'